In [1]:
import asyncio
import json
import os
from typing import List, Dict, Any
from enum import Enum

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from graphiti_core import Graphiti


In [2]:
# Simple context quality assessment
class ContextQuality(Enum):
    INSUFFICIENT = "insufficient"
    SUFFICIENT = "sufficient"
    EXCESSIVE = "excessive"

In [3]:
class SimpleContextAgent:
    def __init__(self):
        # Setup
        self.llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        self.client = Graphiti(
            os.environ.get('NEO4J_URI', 'bolt://localhost:7687'),
            os.environ.get('NEO4J_USER', 'neo4j'),
            os.environ.get('NEO4J_PASSWORD', 'password')
        )
        
    async def get_context(self, query: str, max_results: int = 5) -> tuple[List[str], List[Dict]]:
        """Get context from knowledge base and return both facts and detailed results"""
        try:
            print(f"\n🔍 SEARCHING GRAPH DATABASE:")
            print(f"   Query: '{query}'")
            print(f"   Max Results: {max_results}")
            
            results = await self.client.search(query, num_results=max_results)
            
            # Extract facts and detailed information
            facts = []
            detailed_results = []
            
            print(f"\n📊 GRAPH DATABASE RESULTS ({len(results)} items found):")
            print("-" * 60)
            
            for i, edge in enumerate(results, 1):
                fact = edge.fact
                facts.append(fact)
                
                # Create detailed result info
                result_info = {
                    'index': i,
                    'fact': fact,
                    'confidence': getattr(edge, 'confidence', 'N/A'),
                    'source_node': getattr(edge, 'source_node', 'N/A'),
                    'target_node': getattr(edge, 'target_node', 'N/A'),
                    'edge_type': getattr(edge, 'edge_type', 'N/A'),
                    'metadata': getattr(edge, 'metadata', {})
                }
                detailed_results.append(result_info)
                
                print(f"{i:2d}. Fact: {fact}")
                print(f"    Confidence: {result_info['confidence']}")
                print(f"    Source: {result_info['source_node']}")
                print(f"    Target: {result_info['target_node']}")
                print(f"    Type: {result_info['edge_type']}")
                if result_info['metadata']:
                    print(f"    Metadata: {result_info['metadata']}")
                print()
            
            if not results:
                print("   ❌ No results found in graph database")
            
            return facts, detailed_results
            
        except Exception as e:
            print(f"   ❌ Error searching graph database: {str(e)}")
            return [], []
    
    async def assess_context_quality(self, query: str, context: List[str]) -> Dict[str, Any]:
        """Check if context is sufficient, insufficient, or excessive"""
        
        print(f"\n🔍 ASSESSING CONTEXT QUALITY:")
        print(f"   Query: '{query}'")
        print(f"   Context items: {len(context)}")
        
        assessment_prompt = f"""
        Query: {query}
        
        Context Facts:
        {chr(10).join([f"- {fact}" for fact in context]) if context else "No context provided"}
        
        Assess the context quality for answering this query. Respond in JSON:
        {{
            "quality": "insufficient|sufficient|excessive",
            "confidence": 0.0-1.0,
            "reasoning": "brief explanation",
            "can_answer": true/false,
            "missing_info": "what key information is missing (if any)",
            "relevance_score": 0.0-1.0
        }}
        
        Guidelines:
        - insufficient: Missing key information needed to answer
        - sufficient: Has enough relevant information to answer well
        - excessive: Too much irrelevant information that clutters the response
        """
        
        try:
            print("   📤 Sending assessment prompt to LLM...")
            response = await self.llm.ainvoke([HumanMessage(content=assessment_prompt)])
            assessment = json.loads(response.content)
            
            print(f"   📥 LLM Assessment Result:")
            print(f"      Quality: {assessment.get('quality', 'unknown')}")
            print(f"      Confidence: {assessment.get('confidence', 0):.2f}")
            print(f"      Can Answer: {assessment.get('can_answer', False)}")
            print(f"      Reasoning: {assessment.get('reasoning', 'N/A')}")
            print(f"      Missing Info: {assessment.get('missing_info', 'N/A')}")
            print(f"      Relevance Score: {assessment.get('relevance_score', 0):.2f}")
            
            return assessment
            
        except Exception as e:
            print(f"   ❌ Error in LLM assessment: {str(e)}")
            # Fallback simple assessment
            fallback = {
                "quality": "insufficient" if not context else ("excessive" if len(context) > 10 else "sufficient"),
                "confidence": 1.0 if not context else (0.7 if len(context) > 10 else 0.8),
                "reasoning": "No context" if not context else ("Too many facts" if len(context) > 10 else "Reasonable amount"),
                "can_answer": bool(context),
                "missing_info": "No context available" if not context else "N/A",
                "relevance_score": 0.0 if not context else 0.7
            }
            print(f"   🔄 Using fallback assessment: {fallback}")
            return fallback
    
    async def optimize_context(self, query: str, initial_context: List[str], detailed_results: List[Dict]) -> tuple[List[str], List[Dict]]:
        """Get the right amount of context - not too much, not too little"""
        
        print(f"\n🔧 OPTIMIZING CONTEXT:")
        print(f"   Initial context items: {len(initial_context)}")
        
        if not initial_context:
            print("   📈 No initial context - trying broader search...")
            # Try broader search
            broader_context, broader_details = await self.get_context(query, max_results=10)
            optimized_context = broader_context[:5]  # Cap at 5 for simplicity
            optimized_details = broader_details[:5]
            print(f"   ✅ Broader search returned {len(broader_context)} items, using first 5")
            return optimized_context, optimized_details
        
        # Filter most relevant context
        print("   🎯 Filtering most relevant context using LLM...")
        filter_prompt = f"""
        Query: {query}
        
        Available Context:
        {chr(10).join([f"{i+1}. {fact}" for i, fact in enumerate(initial_context)])}
        
        Select the 3-5 most relevant facts that directly help answer the query.
        Consider relevance, completeness, and avoiding redundancy.
        Return only the numbers separated by commas (e.g., "1,3,5"):
        """
        
        try:
            print("   📤 Sending filter prompt to LLM...")
            response = await self.llm.ainvoke([HumanMessage(content=filter_prompt)])
            selected_nums = [int(n.strip()) for n in response.content.split(',') if n.strip().isdigit()]
            
            print(f"   📥 LLM selected indices: {selected_nums}")
            
            # Filter both facts and detailed results
            optimized_context = [initial_context[i-1] for i in selected_nums if 0 < i <= len(initial_context)]
            optimized_details = [detailed_results[i-1] for i in selected_nums if 0 < i <= len(detailed_results)]
            
            print(f"   ✅ Optimized to {len(optimized_context)} most relevant items")
            
            # Print selected items
            for i, (fact, detail) in enumerate(zip(optimized_context, optimized_details), 1):
                print(f"   {i}. Selected: {fact[:100]}...")
            
            return optimized_context, optimized_details
            
        except Exception as e:
            print(f"   ❌ Error in context optimization: {str(e)}")
            print("   🔄 Using fallback: first 5 items")
            return initial_context[:5], detailed_results[:5]
    
    async def answer_with_context(self, query: str) -> Dict[str, Any]:
        """Main method: Get context, assess quality, and answer"""
        
        print(f"\n{'='*80}")
        print(f"🚀 PROCESSING QUERY: '{query}'")
        print(f"{'='*80}")
        
        # Step 1: Get initial context
        print(f"\n📝 STEP 1: INITIAL CONTEXT RETRIEVAL")
        initial_context, initial_details = await self.get_context(query)
        
        # Step 2: Assess context quality
        print(f"\n📝 STEP 2: INITIAL CONTEXT ASSESSMENT")
        assessment = await self.assess_context_quality(query, initial_context)
        
        # Step 3: Optimize context if needed
        print(f"\n📝 STEP 3: CONTEXT OPTIMIZATION")
        if assessment["quality"] == "insufficient":
            print("   🔄 Context insufficient - expanding search...")
            expanded_context, expanded_details = await self.get_context(query, max_results=10)
            optimized_context, optimized_details = await self.optimize_context(query, expanded_context, expanded_details)
        elif assessment["quality"] == "excessive":
            print("   🔄 Context excessive - filtering to most relevant...")
            optimized_context, optimized_details = await self.optimize_context(query, initial_context, initial_details)
        else:
            print("   ✅ Context quality sufficient - using as is")
            optimized_context = initial_context
            optimized_details = initial_details
        
        # Step 4: Final assessment
        print(f"\n📝 STEP 4: FINAL CONTEXT ASSESSMENT")
        final_assessment = await self.assess_context_quality(query, optimized_context)
        
        # Step 5: Generate answer
        print(f"\n📝 STEP 5: ANSWER GENERATION")
        if final_assessment["can_answer"]:
            print("   ✅ Sufficient context available - generating answer...")
            
            # Print final context being used
            print(f"\n📋 FINAL CONTEXT BEING USED FOR ANSWER:")
            for i, (fact, detail) in enumerate(zip(optimized_context, optimized_details), 1):
                print(f"   {i}. {fact}")
                print(f"      └─ Confidence: {detail.get('confidence', 'N/A')}")
                print(f"      └─ Source: {detail.get('source_node', 'N/A')} → {detail.get('target_node', 'N/A')}")
            
            answer_prompt = f"""
            Query: {query}
            
            Context:
            {chr(10).join([f"- {fact}" for fact in optimized_context])}
            
            Provide a helpful answer based on the context above. Be specific about which facts you're using.
            """
            
            print("   📤 Sending answer generation prompt to LLM...")
            answer_response = await self.llm.ainvoke([HumanMessage(content=answer_prompt)])
            answer = answer_response.content
            print(f"   📥 Answer generated ({len(answer)} characters)")
        else:
            print("   ❌ Insufficient context - cannot provide reliable answer")
            answer = "I don't have enough relevant information to answer your question properly."
        
        # Compile final result
        result = {
            "query": query,
            "initial_context_count": len(initial_context),
            "final_context_count": len(optimized_context),
            "context_quality": final_assessment["quality"],
            "confidence": final_assessment["confidence"],
            "reasoning": final_assessment["reasoning"],
            "context_facts": optimized_context,
            "detailed_context": optimized_details,
            "initial_assessment": assessment,
            "final_assessment": final_assessment,
            "answer": answer
        }
        
        print(f"\n🎉 PROCESSING COMPLETE")
        print(f"   Final Context Quality: {result['context_quality']}")
        print(f"   Confidence: {result['confidence']:.2f}")
        print(f"   Context Items Used: {result['final_context_count']}")
        
        return result


In [4]:
# Tool for external use
@tool
async def smart_search(query: str) -> str:
    """Search with intelligent context optimization"""
    agent = SimpleContextAgent()
    result = await agent.answer_with_context(query)
    
    return f"""
    Context Quality: {result['context_quality']} (confidence: {result['confidence']:.2f})
    Context Used: {result['final_context_count']} facts
    Reasoning: {result['reasoning']}
    
    Answer: {result['answer']}
    """

In [5]:
agent = SimpleContextAgent()

# Test with different types of queries
queries = [
    "What are the best running shoes?",
    "Tell me about customer service policies",
    "How do I return a product?"
]

for query in queries:
    result = await agent.answer_with_context(query)
    
    print(f"\n{'🎯 FINAL SUMMARY':=^80}")
    print(f"Query: {result['query']}")
    print(f"Initial Context: {result['initial_context_count']} facts")
    print(f"Final Context: {result['final_context_count']} facts")
    print(f"Quality: {result['context_quality']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Reasoning: {result['reasoning']}")
    print(f"\nDetailed Context Used:")
    for detail in result['detailed_context']:
        print(f"  • {detail['fact'][:100]}...")
    print(f"\nFinal Answer:\n{result['answer']}")
    print(f"{'='*80}")



🚀 PROCESSING QUERY: 'What are the best running shoes?'

📝 STEP 1: INITIAL CONTEXT RETRIEVAL

🔍 SEARCHING GRAPH DATABASE:
   Query: 'What are the best running shoes?'
   Max Results: 5

📊 GRAPH DATABASE RESULTS (5 items found):
------------------------------------------------------------
 1. Fact: Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 2. Fact: Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 3. Fact: TinyBirds Wool Runners - Little Kids - Natural Black (Blizzard Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 4. Fact: TinyBirds Wool Runners - Little Kids - Natural Black (Blizzard Sole) is a type of Shoes.
    Confidence: N/A
    Source: N/A
    Target: N/A
    Type: N/A

 5. Fact: TinyBirds Wool Runners - Little Kids - Natural Black 

<coroutine object demo at 0x0000022959465240>